# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [30]:
orders = pd.read_csv('Orders.zip')
display(orders.sample(1))
# orders.shape

# There are 4339 individual customerID, 
orders['CustomerID'].value_counts()
#top 5 recurrent customers n`(17841,14911,14096,12748,14606). (Buys most often)
display(orders['CustomerID'].value_counts().head())
#top 5 customers highest spend n`(14646,18102,17450,16446,14911).
display(orders.groupby('CustomerID').agg({'amount_spent':'sum'}).sort_values(by='amount_spent',ascending=False).head())

#Finding the 75% of the aggregated spend by customers:
agg =orders.groupby('CustomerID').agg({'amount_spent':'sum'}).sort_values(by='amount_spent',ascending=False)
display(agg.describe())
#Finding the 95% of the aggregated spend by customers:
display(np.percentile(agg,95))
#Total revenue:
display(agg.sum())

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
388078,526139,580652,21286,2011,12,1,13,retrospot candle large,36,2011-12-05 13:33:00,0.59,17511,United Kingdom,21.24


17841    7847
14911    5677
14096    5111
12748    4596
14606    2700
Name: CustomerID, dtype: int64

,amount_spent
CustomerID,
14646,280206.02
18102,259657.30
17450,194550.79
16446,168472.50
14911,143825.06


,amount_spent
count,4339.000000
mean,2053.793018
std,8988.248381
min,0.000000
25%,307.245000
50%,674.450000
75%,1661.640000
max,280206.020000


5840.181999999983

amount_spent    8911407.904
dtype: float64

In [142]:
#Calculating how many customers there are in VIP customers and Preferred customers categories:
VIP_customers= agg >= np.percentile(agg,95)
Preferred_customers= (agg < np.percentile(agg,95)) & (agg >= np.percentile(agg,75))

display(VIP_customers.value_counts())
display(Preferred_customers.value_counts())

#Creating a new DF with the total spend per customer and merging it to the original DF of orders:
# total_spent = orders.groupby('CustomerID').agg({'amount_spent':'sum'})

# orders = pd.merge(left = orders,
#                   right = total_spent,
#                   how = 'left', 
#                   left_on = "CustomerID", 
#                   right_on= "CustomerID")

#Rename columns:
orders = orders.rename(columns={'amount_spent_y': 'Total Spend', 'amount_spent_x': 'Amount Spent'})

#Creating columns to show if the customer belongs to the VIP or Preferred segment:
orders['VIP_customers'] = orders['Total Spend'] >= np.percentile(agg,95)
orders['Preferred_customers'] = (orders['Total Spend'] < np.percentile(agg,95)) & (orders['Total Spend'] >= np.percentile(agg,75))
display(orders.sample(2))

#Check if both columns have mapped correctly without NaN values
orders['VIP_customers'].value_counts(dropna=False)
orders['Preferred_customers'].value_counts(dropna=False)

#Display CustomerID's that belong to VIP Customers:
display(orders['CustomerID'][orders['Total Spend'] >=np.percentile(agg,95)].unique())
#Display all order numbers of VIP clients:
display(orders['VIP_customers'][orders['VIP_customers'] == True].index)


#To display all orders by a specific CustomerID:
# display(orders[orders['CustomerID'] == 12346])

#To calculate % of Nan values:
# round(orders['VIP_customers'].isna().sum()/len(orders['VIP_customers']),4)*100

amount_spent
False           4122
True             217
dtype: int64

amount_spent
False           3471
True             868
dtype: int64

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Amount Spent,Total Spend,VIP_customers,Preferred_customers
253101,354475,567880,23507,2011,9,4,14,mini playing cards buffalo bill,20,2011-09-22 14:56:00,0.42,15398,United Kingdom,8.40,1345.53,False,False
343949,466315,576326,21584,2011,11,1,15,retrospot small tube matches,3,2011-11-14 15:18:00,1.65,16504,United Kingdom,4.95,484.38,False,False


array([12583, 15311, 16029, 12431, 17511, 13408, 13767, 15513, 13694,
       14849, 16210, 12748, 12433, 14911, 17841, 13093, 12921, 13777,
       18229, 14606, 13576, 13090, 15694, 17017, 15601, 13418, 14060,
       17381, 17581, 15061, 15640, 14031, 12971, 13798, 17396, 14156,
       14680, 12557, 16013, 17949, 12682, 15769, 13081, 17243, 15465,
       13089, 16033, 18055, 18109, 16839, 16814, 12567, 16353, 14527,
       15023, 12472, 16422, 15502, 17677, 17428, 15039, 15078, 14667,
       15194, 17450, 12681, 17735, 15838, 14733, 13488, 17675, 18102,
       13078, 12709, 16779, 14796, 13199, 17706, 16525, 16558, 15498,
       14051, 16713, 13113, 12766, 15005, 14866, 17340, 18092, 15358,
       13319, 12621, 12683, 13854, 17857, 15856, 13102, 13969, 12471,
       12731, 16656, 14952, 12989, 17865, 16873, 14062, 16923, 12753,
       13668, 15044, 14505, 12540, 13225, 13209, 17338, 12476, 15159,
       13324, 14961, 14057, 14298, 17404, 14415, 13097, 13458, 15290,
       15615, 15482,

Int64Index([    26,     27,     28,     29,     30,     31,     32,     33,
                34,     35,
            ...
            397878, 397879, 397880, 397881, 397882, 397883, 397905, 397906,
            397907, 397908],
           dtype='int64', length=104484)

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [ ]:
# see above

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [145]:
orders.groupby('Country').agg({'VIP_customers':'sum'}).sort_values(by='VIP_customers', ascending=False)

,VIP_customers
Country,
United Kingdom,84185
EIRE,7077
France,3290
Germany,3127
Netherlands,2080
Australia,898
Portugal,681
Switzerland,594
Spain,511


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [189]:
orders.pivot_table(index=['Country'], values=['VIP_customers', 'Preferred_customers'], aggfunc=['sum']).sort_values(by=[(    'sum', 'VIP_customers'),
            (    'sum',       'Preferred_customers')],ascending=False)


sum              
                     Preferred_customers VIP_customers
Country                                               
United Kingdom                    137450         84185
EIRE                                 161          7077
France                              3011          3290
Germany                             4222          3127
Netherlands                            0          2080
Australia                            130           898
Portugal                             412           681
Switzerland                          776           594
Spain                               1058           511
Norway                               608           420
Channel Islands                      225           364
Finland                              210           294
Cyprus                               203           248
Singapore                              0           222
Japan                                 67           205
Sweden                                75           198
Belgium                             1503            54
Denmark                              181            36
Italy                                507             0
Israel                               214             0
Iceland                              182             0
Austria                              158             0
Poland                               149             0
Canada                               135             0
Malta                                 67             0
Lebanon                               45             0
Greece                                32             0
Bahrain                                0             0
Brazil                                 0             0
Czech Republic                         0             0
European Community                     0             0
Lithuania                              0             0
RSA                                    0             0
Saudi Arabia                           0             0
USA                                    0             0
United Arab Emirates                   0             0
Unspecified                            0             0